# Crear Base efectos secuenciales

## Funciones

In [1]:
# Librerías de siempre
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import os
import statsmodels.api as sm

from statsmodels.formula.api import ols
from scipy import stats

In [2]:
def formatear_base(db):
    df = db.copy().reset_index(drop=True)
    if 'fecha' in df.columns:
        df = df.sort_values(by=['Torneo','Round'], ascending = [True,True]).reset_index(drop=True)
    equipos = df['Local'].unique()
    equipos = pd.DataFrame(equipos, columns=['equipo'])
    equipos['i'] = equipos.index
    df = pd.merge(df, equipos, left_on='Local', right_on='equipo', how='left')
    df = df.rename(columns = {'i': 'i_local'}).drop('equipo', 1)
    df = pd.merge(df, equipos, left_on='Visita', right_on='equipo', how='left')
    df = df.rename(columns = {'i': 'i_visita'}).drop('equipo', 1)
    df = df.replace({'. Round' : ''}, regex = True)
    df['Round'] = df['Round'].astype(int)
    df['goles L'] = df['goles L'].astype(int)
    df['goles V'] = df['goles V'].astype(int)
    return df

In [3]:
def tabla_final_torneo(db):
    df = db.copy().reset_index(drop=True)
    if 'i_local' not in df.columns:
        df = formatear_base(df)
    tabla = df[['Local','i_local']].drop_duplicates()
    tabla = tabla.set_index(['i_local'])
    tabla.columns = ['equipo']
    conditions = [
            (df['goles L'] > df['goles V']),
            (df['goles L'] < df['goles V'])]
    choices = ['local', 'visita']
    df = df.join(pd.get_dummies(np.select(conditions, choices, default = 'empate')))
    ghome = df.groupby('i_local')
    gaway = df.groupby('i_visita')
    df_home = pd.DataFrame({'wins_h': ghome['local'].sum(),
                            'draws_h': ghome['empate'].sum(),
                            'losses_h': ghome['visita'].sum(),
                            'gf_h': ghome['goles L'].sum(),
                            'ga_h': ghome['goles V'].sum(),
                            'gd_h': ghome['goles L'].sum() - ghome['goles V'].sum()})
    df_away = pd.DataFrame({'wins_a': gaway['visita'].sum(),
                            'draws_a': gaway['empate'].sum(),
                            'losses_a': gaway['local'].sum(),
                            'gf_a': gaway['goles V'].sum(),
                            'ga_a': gaway['goles L'].sum(),
                            'gd_a': gaway['goles V'].sum() - gaway['goles L'].sum()})
    tabla = tabla.join(df_home, how='left').join(df_away,how = 'left').fillna(0)
    tabla['wins'] = tabla.wins_h + tabla.wins_a
    tabla['draws'] = tabla.draws_h + tabla.draws_a
    tabla['losses'] = tabla.losses_h + tabla.losses_a
    tabla['gf'] = tabla.gf_h + tabla.gf_a
    tabla['ga'] = tabla.ga_h + tabla.ga_a
    tabla['gd'] = tabla.gd_h + tabla.gd_a
    tabla['points'] = (tabla['wins']*3 + tabla['draws']).astype(int)
    tabla = tabla.sort_values(by=['points','gd'], ascending = False).reset_index(drop=True)
    tabla['position'] = (tabla.index + 1).astype(int)
    return tabla[['equipo','wins','draws','losses','gf','ga','gd','points','position']]

In [4]:
def tabla_final_local(db):
    df = db.copy().reset_index(drop=True)
    if 'i_local' not in df.columns:
        df = formatear_base(df)
    tabla = df[['Local','i_local']].drop_duplicates()
    tabla = tabla.set_index(['i_local'])
    tabla.columns = ['equipo']
    conditions = [
            (df['goles L'] > df['goles V']),
            (df['goles L'] < df['goles V'])]
    choices = ['local', 'visita']
    df = df.join(pd.get_dummies(np.select(conditions, choices, default = 'empate')))
    ghome = df.groupby('i_local')
    df_home = pd.DataFrame({'wins': ghome['local'].sum(),
                            'draws': ghome['empate'].sum(),
                            'losses': ghome['visita'].sum(),
                            'gf': ghome['goles L'].sum(),
                            'ga': ghome['goles V'].sum(),
                            'gd': ghome['goles L'].sum() - ghome['goles V'].sum()})
    tabla = tabla.join(df_home, how='left').fillna(0)
    tabla['points'] = (tabla['wins']*3 + tabla['draws']).astype(int)
    #Normalizar puntos según cantidad de partidos jugados de local
    tabla['norm_points'] = tabla['points']/(tabla['wins'] + tabla['draws'] + tabla['losses']) 
    tabla = tabla.sort_values(by=['norm_points','points','gd'], ascending = False).reset_index(drop=True)
    tabla['position'] = (tabla.index + 1).astype(int)
    return tabla[['equipo','wins','draws','losses','gf','ga','gd','points','position']]

In [5]:
def tabla_final_visita(db):
    df = db.copy()
    if 'i_visita' not in df.columns:
        df = formatear_base(df)
    tabla = df[['Visita','i_visita']].drop_duplicates()
    tabla = tabla.set_index(['i_visita'])
    tabla.columns = ['equipo']
    conditions = [
            (df['goles L'] > df['goles V']),
            (df['goles L'] < df['goles V'])]
    choices = ['local', 'visita']
    df = df.join(pd.get_dummies(np.select(conditions, choices, default = 'empate')))
    gaway = df.groupby('i_visita')
    df_away = pd.DataFrame({'wins': gaway['visita'].sum(),
                            'draws': gaway['empate'].sum(),
                            'losses': gaway['local'].sum(),
                            'gf': gaway['goles V'].sum(),
                            'ga': gaway['goles L'].sum(),
                            'gd': gaway['goles V'].sum() - gaway['goles L'].sum()})
    tabla = tabla.join(df_away, how='left').fillna(0)
    tabla['points'] = (tabla['wins']*3 + tabla['draws']).astype(int)
    #Normalizar puntos según cantidad de partidos jugados de local
    tabla['norm_points'] = tabla['points']/(tabla['wins'] + tabla['draws'] + tabla['losses']) 
    tabla = tabla.sort_values(by=['norm_points','points','gd'], ascending = False).reset_index(drop=True)
    tabla['position'] = (tabla.index + 1).astype(int)
    return tabla[['equipo','wins','draws','losses','gf','ga','gd','points','position']]

In [6]:
def variables_efecto_secuencial(df_actual, ronda_corte = 5, punto_corte = 5):
    r,c = ronda_corte, punto_corte
    torneo = df_actual['Torneo'].drop_duplicates().values[0]
    lista_equipos = df_actual['Local'].drop_duplicates().tolist()
    df_test_pX = df_actual[df_actual['Round'] <= ronda_corte]
    cols_local = ['Visita'] + [i for i in df_actual.columns if '_' in i and 'local' not in i and 'ronda' not in i]
    func_local = ['count'] + ['mean' if i.startswith('posicion') else 'sum' for i in cols_local[1:]]
    cols_visita = ['Local'] + [i for i in df_actual.columns if '_' in i and 'visita' not in i and 'ronda' not in i]
    func_visita = ['count'] + ['mean' if i.startswith('posicion') else 'sum' for i in cols_visita[1:]]
    dict_df_local = dict(zip(cols_local,func_local))
    dict_df_visita = dict(zip(cols_visita,func_visita))
    g_test_l = df_test_pX.groupby('Local').agg(dict_df_local)
    g_test_l.columns = ['partidos_local'] + cols_local[1:]
    if len(g_test_l.index.tolist()) != len(lista_equipos):
        parche = [e for e in lista_equipos if e not in g_test_l.index.tolist()]
        for e in parche:
            g_test_l.loc[e] = [0 for c in g_test_l.columns]
    g_test_v = df_test_pX.groupby('Visita').agg(dict_df_visita)
    g_test_v.columns = ['partidos_visita'] + cols_visita[1:]
    if len(g_test_v.index.tolist()) != len(lista_equipos):
        parche = [e for e in lista_equipos if e not in g_test_v.index.tolist()]
        for e in parche:
            g_test_v.loc[e] = [0 for c in g_test_v.columns]
#     cols_ronda_local = [i for i in df_actual.columns if 'ronda' in i and 'local' not in i]
#     func_ronda = ['min' for i in cols_ronda_local]
#     g_test_ronda_local = df_actual.groupby('Local').agg(dict(zip(cols_ronda_local,func_ronda)))
#     cols_ronda_visita = [i for i in df_actual.columns if 'ronda' in i and 'visita' not in i]
#     g_test_ronda_visita = df_actual.groupby('Visita').agg(dict(zip(cols_ronda_visita,func_ronda)))
#     g_test_ronda = g_test_ronda_local.join(g_test_ronda_visita)
    g_test_lv = g_test_l.join(g_test_v)
    """
    SIGNIFICADO VARIABLES
    
    total_partidos: total de partidos disputados en las primeras X rondas
    
    facil_general: cantidad de partidos fáciles considerando la tabla general del
    torneo anterior
    perc_facil_general: porcentaje partidos fáciles considerando la tabla general
    del torneo anterior
    dificil_general: cantidad de partidos difíciles considerando la tabla general
    del torneo anterior
    perc_dificil_general: porcentaje partidos difíciles considerando la tabla general
    del torneo anterior
    
    facil_lv: cantidad de partidos fáciles considerando las tablas de local y visita
    del torneo anterior
    perc_facil_lv: porcentaje partidos fáciles considerando las tablas de local y visita
    del torneo anterior
    dificil_lv: cantidad de partidos difíciles considerando las tablas de local y visita
    del torneo anterior
    perc_dificil_lv: porcentaje partidos difíciles considerando las tablas de local y visita
    del torneo anterior
    
    prom_posicion_general: promedio de posición de rivales considerando tabla general
    del torneo anterior
    prom_posicion_lv: promedio de posición de rivales considerando las tablas de local y visita
    del torneo anterior
    
    ronda_primer_facil_general: ronda en que juega contra el primer equipo fácil según la tabla general
    del torneo anterior
    ronda_primer_dificil_general: ronda en que juega contra el primer equipo difícil según la tabla general
    del torneo anterior
    ronda_primer_facil_lv: ronda en que juega contra el primer equipo fácil según las tablas de local y visita
    del torneo anterior
    ronda_primer_dificil_lv: ronda en que juega contra el primer equipo difícil según las tablas de local y visita
    del torneo anterior
    
    """
    g_test_lv['perc_facil_elo_p%s_c%s' % (r,c)] = (g_test_lv['f_elo_local'] + g_test_lv['f_elo_visita'])/r
    g_test_lv['perc_dificil_elo_p%s_c%s' % (r,c)] = (g_test_lv['d_elo_local'] + g_test_lv['d_elo_visita'])/r
#     g_test_lv['total_partidos'] = g_test_lv['partidos_local'] + g_test_lv['partidos_visita']
    
#     g_test_lv['facil_general'] = g_test_lv['f_gral_local'] + g_test_lv['f_gral_visita']
#     g_test_lv['perc_facil_general_p%s_c%s' % (r,c)] = g_test_lv['facil_general']/g_test_lv['total_partidos']
#     g_test_lv['dificil_general'] = g_test_lv['d_gral_local'] + g_test_lv['d_gral_visita']
#     g_test_lv['perc_dificil_general_p%s_c%s' % (r,c)] = g_test_lv['dificil_general']/g_test_lv['total_partidos']
#     g_test_lv['perc_regular_general_p%s_c%s' % (r,c)] = 1 - g_test_lv['perc_dificil_general_p%s_c%s' % (r,c)] - g_test_lv['perc_facil_general_p%s_c%s' % (r,c)]
    
#     g_test_lv['facil_lv'] = g_test_lv['f_local'] + g_test_lv['f_visita']
#     g_test_lv['perc_facil_lv_p%s_c%s' % (r,c)] = g_test_lv['facil_lv']/g_test_lv['total_partidos']
#     g_test_lv['dificil_lv'] = g_test_lv['d_local'] + g_test_lv['d_visita']
#     g_test_lv['perc_dificil_lv_p%s_c%s' % (r,c)] = g_test_lv['dificil_lv']/g_test_lv['total_partidos']
#     g_test_lv['perc_regular_lv_p%s_c%s' % (r,c)] = 1 - g_test_lv['perc_dificil_lv_p%s_c%s' % (r,c)] - g_test_lv['perc_facil_lv_p%s_c%s' % (r,c)]
    
    
    g_test_lv = g_test_lv[[i for i in g_test_lv if i not in cols_local and i not in cols_visita and 'partidos' not in i]]
    g_test_lv = g_test_lv[[i for i in g_test_lv if not i.endswith('general') and not i.endswith('lv')]]
#     g_test_ronda = g_test_ronda[[i for i in g_test_ronda if i not in cols_ronda_local and i not in cols_ronda_visita]]
    t_act = tabla_final_torneo(df_actual)[['equipo','position','points']]
    t_act.columns = ['equipo','position_final','points_final']
    # Aquí está OK
    ronda_mitad = df_actual['Round'].max()/2
    t_act_mitad = tabla_final_torneo(df_actual[df_actual['Round'] <= ronda_mitad])[['equipo','position','points']]
    t_act_mitad.columns = ['equipo','position_mitad','points_mitad']
    t_act = pd.merge(t_act, t_act_mitad, left_on = 'equipo', right_on = 'equipo')
    #Aquí está Ok
    t_act_l = tabla_final_local(df_actual)[['equipo','position','points']]
    t_act_l.columns = ['equipo','position_l_final','points_l_final']
    t_act_l_mitad = tabla_final_local(df_actual[df_actual['Round'] <= ronda_mitad])[['equipo','position','points']]
    t_act_l_mitad.columns = ['equipo','position_l_mitad','points_l_mitad']
    t_act_l = pd.merge(t_act_l, t_act_l_mitad, left_on = 'equipo', right_on = 'equipo')
    # Aquí está Ok
    t_act_v = tabla_final_visita(df_actual)[['equipo','position','points']]
    t_act_v.columns = ['equipo','position_v_final','points_v_final']
    t_act_v_mitad = tabla_final_visita(df_actual[df_actual['Round'] <= ronda_mitad])[['equipo','position','points']]
    t_act_v_mitad.columns = ['equipo','position_v_mitad','points_v_mitad']
    t_act_v = pd.merge(t_act_v, t_act_v_mitad, left_on = 'equipo', right_on = 'equipo')
    # Aquí está Ok
    t_act_lv = pd.merge(t_act_l, t_act_v, left_on = 'equipo', right_on = 'equipo')
    t_act = pd.merge(t_act, t_act_lv, left_on = 'equipo', right_on = 'equipo')
    # WARNING
#     if r == 3 and c == 3:
#         print(t_act['equipo'].tolist())
    t_act['Torneo'] = torneo
    df_efecto = pd.merge(t_act, g_test_lv, left_on = 'equipo', right_index = True)
#     df_efecto = pd.merge(t_act, g_test_lv.join(g_test_ronda), left_on = 'equipo', right_index = True)
    # Aquí se pierde un equipo en la temp 2017/18
#     if r == 3 and c == 3:
#         print(df_efecto['equipo'].tolist())
    return df_efecto

In [7]:
# RESPALDO
# def indicadores_torneo(df,torneo_actual,torneo_anterior, replace_releg = False):
#     """
#     Crea las columnas que se agregarán para definir los indicadores de interés
#     """
#     t_ant = tabla_final_torneo(df[df['Torneo'] == torneo_anterior])
#     t_ant_local = tabla_final_local(df[df['Torneo'] == torneo_anterior])
#     t_ant_visita = tabla_final_visita(df[df['Torneo'] == torneo_anterior])
#     df_actual = formatear_base(df[df['Torneo'] == torneo_actual])[['Local','Visita','goles L','goles V','Torneo','Round']].sort_values(by=['Round'], ascending = True).reset_index(drop=True)
#     eq_act = df_actual['Local'].drop_duplicates().tolist()
#     eq_ant_l =  t_ant_local['equipo'].drop_duplicates().tolist()
#     eq_ant_v =  t_ant_visita['equipo'].drop_duplicates().tolist()
#     eq_ant = t_ant['equipo'].drop_duplicates().tolist() #esta ya está ordenada por posición
#     eq_correc = [i for i in eq_ant if i in eq_act] + [j for j in eq_act if j not in eq_ant]
#     eq_correc_l = [i for i in eq_ant_l if i in eq_act] + [j for j in eq_act if j not in eq_ant_l]
#     eq_correc_v = [i for i in eq_ant_v if i in eq_act] + [j for j in eq_act if j not in eq_ant_v]
#     # reemplazar o eliminar equipos descendidos
#     if replace_releg:
#         t_ant = pd.DataFrame({'equipo': eq_correc,
#                               'position': [i+1 for i in range(len(eq_correc))]})
#         t_ant_local = pd.DataFrame({'equipo': eq_correc_l,
#                               'position': [i+1 for i in range(len(eq_correc_l))]})
#         t_ant_visita = pd.DataFrame({'equipo': eq_correc_v,
#                               'position': [i+1 for i in range(len(eq_correc_v))]})
#     else:    
#         t_ant_local = t_ant_local[t_ant_local['equipo'].isin(eq_act)].reset_index(drop=True)
#         t_ant_local['position'] = t_ant_local.index + 1
#         t_ant_visita = t_ant_visita[t_ant_visita['equipo'].isin(eq_act)].reset_index(drop=True)
#         t_ant_visita['position'] = t_ant_visita.index + 1
#         posicion_nuevos = t_ant_local['position'].max() + 1
#         position = [min(i+1,posicion_nuevos) for i in range(len(eq_correc))]
#         t_ant = pd.DataFrame({'equipo': eq_correc,
#                           'position': position})
#         t_ant_local = pd.DataFrame({'equipo': eq_correc_l,
#                               'position': position})
#         t_ant_visita = pd.DataFrame({'equipo': eq_correc_v,
#                               'position': position})
#     t_ant_ultimo = t_ant['position'].astype(int).max()
#     cols_main = ['equipo', 'position', 'Torneo']
#     dfs_efecto = []
#     for c in [3,4,5]:
#         pto_corte = c
#         equipos_5p = t_ant[t_ant['position'] <= pto_corte]['equipo'].values.tolist()
#         local_5p = t_ant_local[t_ant_local['position'] <= pto_corte]['equipo'].values.tolist()
#         visita_5p = t_ant_visita[t_ant_visita['position'] <= pto_corte]['equipo'].values.tolist()
#         equipos_5u = t_ant[t_ant['position'] >= t_ant_ultimo - pto_corte + 1]['equipo'].values.tolist()
#         local_5u = t_ant_local[t_ant_local['position'] >= t_ant_ultimo - pto_corte + 1]['equipo'].values.tolist() 
#         visita_5u = t_ant_visita[t_ant_visita['position'] >= t_ant_ultimo - pto_corte + 1]['equipo'].values.tolist()

#         dict_eq_pos_gral = dict(zip(t_ant['equipo'].values, t_ant['position'].values))
#         dict_eq_pos_local = dict(zip(t_ant_local['equipo'].values, t_ant_local['position'].values))
#         dict_eq_pos_visita = dict(zip(t_ant_visita['equipo'].values, t_ant_visita['position'].values))


#         """
#         SIGNIFICADO VARIABLES


#         f_gral_visita: 1 si el equipo visitante es fácil según la tabla general
#         del torneo anterior, 0 si no
#         d_gral_visita: 1 si el equipo visitante es difícil según la tabla general
#         del torneo anterior, 0 si no
#         f_gral_local: 1 si el equipo local es fácil según la tabla general
#         del torneo anterior, 0 si no
#         d_gral_local: 1 si el equipo local es difícil según la tabla general
#         del torneo anterior, 0 si no

#         f_visita: 1 si el equipo visitante es fácil jugando de visita según
#         la tabla de posiciones de visita, 0 si no
#         d_visita: 1 si el equipo visitante es difícil jugando de visita según
#         la tabla de posiciones de visita, 0 si no
#         f_local: 1 si equipo local es fácil jugando de local según
#         la tabla de posiciones de local, 0 si no
#         d_local: 1 si el equipo local es difícil jugando de local según
#         la tabla de posiciones de local, 0 si no

#         f_..._corr es lo mismo, salvo que se hacre la corrección de pertenecer
#         al mismo grupo

#         posicion_gral_visita: posición del equipo visitante según la tabla general del
#         torneo anterior
#         posicion_gral_local: posición del equipo local según la tabla general del
#         torneo anterior
#         posicion_visita: posición del equipo visitante según la tabla de visita del
#         torneo anterior
#         posicion_local: posición del equipo local según la tabla de local del
#         torneo anterior

#         ronda_f_gral_visita: si el equipo visitante es fácil según la tabla general,
#         toma valor igual a la ronda. Si no lo es, toma np.nan
#         ronda_d_gral_visita: si el equipo visitante es difícil según la tabla general,
#         toma valor igual a la ronda. Si no lo es, toma np.nan    
#         ronda_f_gral_local: si el equipo local es fácil según la tabla general,
#         toma valor igual a la ronda. Si no lo es, toma np.nan
#         ronda_d_gral_local: si el equipo local es difícil según la tabla general,
#         toma valor igual a la ronda. Si no lo es, toma np.nan

#         ronda_f_visita: si el equipo visitante es fácil según la tabla de visita,
#         toma valor igual a la ronda. Si no lo es, toma np.nan
#         ronda_d_visita: si el equipo visitante es difícil según la tabla de visita,
#         toma valor igual a la ronda. Si no lo es, toma np.nan    
#         ronda_f_local: si el equipo local es fácil según la tabla de local,
#         toma valor igual a la ronda. Si no lo es, toma np.nan
#         ronda_d_local: si el equipo local es difícil según la tabla de local,
#         toma valor igual a la ronda. Si no lo es, toma np.nan


#         """
    
#         df_actual['f_gral_visita'] = np.where(df_actual['Visita'].isin(equipos_5u), 1, 0)
#         df_actual['d_gral_visita'] = np.where(df_actual['Visita'].isin(equipos_5p), 1, 0)
#         df_actual['f_gral_local'] = np.where(df_actual['Local'].isin(equipos_5u), 1, 0)
#         df_actual['d_gral_local'] = np.where(df_actual['Local'].isin(equipos_5p), 1, 0)    

#         df_actual['f_visita'] = np.where(df_actual['Visita'].isin(visita_5u), 1, 0)
#         df_actual['d_visita'] = np.where(df_actual['Visita'].isin(visita_5p), 1, 0)
#         df_actual['f_local'] = np.where(df_actual['Local'].isin(local_5u), 1, 0)
#         df_actual['d_local'] = np.where(df_actual['Local'].isin(visita_5p), 1, 0)

#         df_actual['f_gral_visita_corr'] =  df_actual['f_gral_visita']*np.where(df_actual['Local'].isin(equipos_5u),pto_corte/(pto_corte - 1) , 1)
#         df_actual['d_gral_visita_corr'] =  df_actual['d_gral_visita']*np.where(df_actual['Local'].isin(equipos_5p),pto_corte/(pto_corte - 1) , 1)
#         df_actual['f_gral_local_corr'] =  df_actual['f_gral_local']*np.where(df_actual['Visita'].isin(equipos_5u),pto_corte/(pto_corte - 1) , 1)
#         df_actual['d_gral_local_corr'] =  df_actual['d_gral_local']*np.where(df_actual['Visita'].isin(equipos_5p),pto_corte/(pto_corte - 1) , 1)

#         df_actual['f_visita_corr'] =  df_actual['f_visita']*np.where(df_actual['Local'].isin(visita_5u),pto_corte/(pto_corte - 1) , 1)
#         df_actual['d_visita_corr'] =  df_actual['d_visita']*np.where(df_actual['Local'].isin(visita_5p),pto_corte/(pto_corte - 1) , 1)
#         df_actual['f_local_corr'] = df_actual['f_local']*np.where(df_actual['Visita'].isin(local_5u),pto_corte/(pto_corte - 1) , 1)
#         df_actual['d_local_corr'] =  df_actual['d_local']*np.where(df_actual['Visita'].isin(local_5p),pto_corte/(pto_corte - 1) , 1)

#         df_actual['posicion_gral_visita'] = df_actual['Visita'].map(dict_eq_pos_gral)
#         df_actual['posicion_gral_local'] = df_actual['Local'].map(dict_eq_pos_gral)
#         df_actual['posicion_visita'] = df_actual['Visita'].map(dict_eq_pos_visita)
#         df_actual['posicion_local'] = df_actual['Local'].map(dict_eq_pos_local)


#         df_actual['ronda_f_gral_visita'] = df_actual['Round']*np.where(df_actual['Visita'].isin(equipos_5u), 1, np.nan)
#         df_actual['ronda_d_gral_visita'] = df_actual['Round']*np.where(df_actual['Visita'].isin(equipos_5p), 1, np.nan)
#         df_actual['ronda_f_gral_local'] = df_actual['Round']*np.where(df_actual['Local'].isin(equipos_5u), 1, np.nan)
#         df_actual['ronda_d_gral_local'] = df_actual['Round']*np.where(df_actual['Local'].isin(equipos_5p), 1, np.nan)

#         df_actual['ronda_f_visita'] = df_actual['Round']*np.where(df_actual['Visita'].isin(visita_5u), 1, np.nan)
#         df_actual['ronda_d_visita'] = df_actual['Round']*np.where(df_actual['Visita'].isin(visita_5p), 1, np.nan)
#         df_actual['ronda_f_local'] = df_actual['Round']*np.where(df_actual['Local'].isin(local_5u), 1, np.nan)
#         df_actual['ronda_d_local'] = df_actual['Round']*np.where(df_actual['Local'].isin(local_5p), 1, np.nan)

#         df_actual['ronda_f_gral_visita'] = df_actual['Round']*np.where(df_actual['Visita'].isin(equipos_5u), 1, np.nan)
#         df_actual['ronda_d_gral_visita'] = df_actual['Round']*np.where(df_actual['Visita'].isin(equipos_5p), 1, np.nan)
#         df_actual['ronda_f_gral_local'] = df_actual['Round']*np.where(df_actual['Local'].isin(equipos_5u), 1, np.nan)
#         df_actual['ronda_d_gral_local'] = df_actual['Round']*np.where(df_actual['Local'].isin(equipos_5p), 1, np.nan)

#         df_actual['ronda_f_visita'] = df_actual['Round']*np.where(df_actual['Visita'].isin(visita_5u), 1, np.nan)
#         df_actual['ronda_d_visita'] = df_actual['Round']*np.where(df_actual['Visita'].isin(visita_5p), 1, np.nan)
#         df_actual['ronda_f_local'] = df_actual['Round']*np.where(df_actual['Local'].isin(local_5u), 1, np.nan)
#         df_actual['ronda_d_local'] = df_actual['Round']*np.where(df_actual['Local'].isin(local_5p), 1, np.nan)
        
#         for r in [4,5,6,7]:
#             df_efecto = variables_efecto_secuencial(df_actual, punto_corte = c, ronda_corte = r)
#             if r == 4:
#                 conditions_g = [
#                         (df_efecto['equipo'].isin(equipos_5p)),
#                         (df_efecto['equipo'].isin(equipos_5u))]
#                 conditions_l = [
#                         (df_efecto['equipo'].isin(local_5p)),
#                         (df_efecto['equipo'].isin(local_5u))]
#                 conditions_v = [
#                         (df_efecto['equipo'].isin(visita_5p)),
#                         (df_efecto['equipo'].isin(visita_5u))]
#                 conditions = [conditions_g, conditions_l, conditions_v]
#                 choices = ['dificil','facil']
#                 newvar_names = ['categoria_general_c%s' % c,
#                                 'categoria_local_c%s' % c,
#                                 'categoria_visita_c%s' % c]
#                 for n in range(len(newvar_names)):
#                     newvar = newvar_names[n]    
#                     condition = conditions[n]
#                     df_efecto[newvar] = np.select(condition,
#                                                   choices,
#                                                   default = 'regular')
#                     df_efecto[newvar] = df_efecto[newvar].astype('category')
#                     df_efecto[newvar].cat.reorder_categories(['regular', 'facil','dificil'], inplace = True)
#                 if c == 3:
#                     dfs_efecto.append(df_efecto)
#                 else:
#                     dfs_efecto.append(df_efecto[[i for i in df_efecto.columns if i not in cols_main]])
#             else:
#                 dfs_efecto.append(df_efecto[[i for i in df_efecto.columns if i not in cols_main]])
            
            
#     df_efecto_final = pd.concat(dfs_efecto, axis = 1)
#     return df_efecto_final

In [8]:
def indicadores_torneo(df,df_elo, torneo_actual,torneo_anterior, replace_releg = False):
    """
    Crea las columnas que se agregarán para definir los indicadores de interés
    """
    t_ant = tabla_final_torneo(df[df['Torneo'] == torneo_anterior])
    t_ant_local = tabla_final_local(df[df['Torneo'] == torneo_anterior])
    t_ant_visita = tabla_final_visita(df[df['Torneo'] == torneo_anterior])
    df_actual = formatear_base(df[df['Torneo'] == torneo_actual])[['Local','Visita','goles L','goles V','Torneo','Round']].sort_values(by=['Round'], ascending = True).reset_index(drop=True)
    eq_act = df_actual['Local'].drop_duplicates().tolist()
    df_elo_actual = df_elo[(df_elo['Torneo'] == torneo_actual) & (df_elo['Club'].isin(eq_act))]
    df_elo_actual = df_elo_actual.sort_values(by = 'Elo',ascending = False).reset_index(drop=True)
    eq_ant_l =  t_ant_local['equipo'].drop_duplicates().tolist()
    eq_ant_v =  t_ant_visita['equipo'].drop_duplicates().tolist()
    eq_ant = t_ant['equipo'].drop_duplicates().tolist() #esta ya está ordenada por posición
    eq_correc = [i for i in eq_ant if i in eq_act] + [j for j in eq_act if j not in eq_ant]
    eq_correc_l = [i for i in eq_ant_l if i in eq_act] + [j for j in eq_act if j not in eq_ant_l]
    eq_correc_v = [i for i in eq_ant_v if i in eq_act] + [j for j in eq_act if j not in eq_ant_v]
    # reemplazar o eliminar equipos descendidos
    if replace_releg:
        t_ant = pd.DataFrame({'equipo': eq_correc,
                              'position': [i+1 for i in range(len(eq_correc))]})
        t_ant_local = pd.DataFrame({'equipo': eq_correc_l,
                              'position': [i+1 for i in range(len(eq_correc_l))]})
        t_ant_visita = pd.DataFrame({'equipo': eq_correc_v,
                              'position': [i+1 for i in range(len(eq_correc_v))]})
    else:    
        t_ant_local = t_ant_local[t_ant_local['equipo'].isin(eq_act)].reset_index(drop=True)
        t_ant_local['position'] = t_ant_local.index + 1
        t_ant_visita = t_ant_visita[t_ant_visita['equipo'].isin(eq_act)].reset_index(drop=True)
        t_ant_visita['position'] = t_ant_visita.index + 1
        posicion_nuevos = t_ant_local['position'].max() + 1
        position = [min(i+1,posicion_nuevos) for i in range(len(eq_correc))]
        t_ant = pd.DataFrame({'equipo': eq_correc,
                          'position': position})
        t_ant_local = pd.DataFrame({'equipo': eq_correc_l,
                              'position': position})
        t_ant_visita = pd.DataFrame({'equipo': eq_correc_v,
                              'position': position})
    t_ant_ultimo = t_ant['position'].astype(int).max()
    cols_main = ['equipo', 'position', 'Torneo']
    dfs_efecto = []
    equipos_elo = df_elo_actual['Club'].tolist()
    if len(equipos_elo) < len(eq_act):
        eq_act = eq_act + [e for e in equipos_elo if e not in eq_act]
#     for c in [3,4,5]:
    for c in [3,4,5,6]:
        pto_corte = c
#         equipos_5p = t_ant[t_ant['position'] <= pto_corte]['equipo'].values.tolist()
#         local_5p = t_ant_local[t_ant_local['position'] <= pto_corte]['equipo'].values.tolist()
#         visita_5p = t_ant_visita[t_ant_visita['position'] <= pto_corte]['equipo'].values.tolist()
#         equipos_5u = t_ant[t_ant['position'] >= t_ant_ultimo - pto_corte + 1]['equipo'].values.tolist()
#         local_5u = t_ant_local[t_ant_local['position'] >= t_ant_ultimo - pto_corte + 1]['equipo'].values.tolist() 
#         visita_5u = t_ant_visita[t_ant_visita['position'] >= t_ant_ultimo - pto_corte + 1]['equipo'].values.tolist()
        elo_5p = equipos_elo[:c]
        elo_5u = equipos_elo[-c:]
        

#         dict_eq_pos_gral = dict(zip(t_ant['equipo'].values, t_ant['position'].values))
#         dict_eq_pos_local = dict(zip(t_ant_local['equipo'].values, t_ant_local['position'].values))
#         dict_eq_pos_visita = dict(zip(t_ant_visita['equipo'].values, t_ant_visita['position'].values))


        """
        SIGNIFICADO VARIABLES


        f_gral_visita: 1 si el equipo visitante es fácil según la tabla general
        del torneo anterior, 0 si no
        d_gral_visita: 1 si el equipo visitante es difícil según la tabla general
        del torneo anterior, 0 si no
        f_gral_local: 1 si el equipo local es fácil según la tabla general
        del torneo anterior, 0 si no
        d_gral_local: 1 si el equipo local es difícil según la tabla general
        del torneo anterior, 0 si no

        f_visita: 1 si el equipo visitante es fácil jugando de visita según
        la tabla de posiciones de visita, 0 si no
        d_visita: 1 si el equipo visitante es difícil jugando de visita según
        la tabla de posiciones de visita, 0 si no
        f_local: 1 si equipo local es fácil jugando de local según
        la tabla de posiciones de local, 0 si no
        d_local: 1 si el equipo local es difícil jugando de local según
        la tabla de posiciones de local, 0 si no

        f_..._corr es lo mismo, salvo que se hacre la corrección de pertenecer
        al mismo grupo

        posicion_gral_visita: posición del equipo visitante según la tabla general del
        torneo anterior
        posicion_gral_local: posición del equipo local según la tabla general del
        torneo anterior
        posicion_visita: posición del equipo visitante según la tabla de visita del
        torneo anterior
        posicion_local: posición del equipo local según la tabla de local del
        torneo anterior

        ronda_f_gral_visita: si el equipo visitante es fácil según la tabla general,
        toma valor igual a la ronda. Si no lo es, toma np.nan
        ronda_d_gral_visita: si el equipo visitante es difícil según la tabla general,
        toma valor igual a la ronda. Si no lo es, toma np.nan    
        ronda_f_gral_local: si el equipo local es fácil según la tabla general,
        toma valor igual a la ronda. Si no lo es, toma np.nan
        ronda_d_gral_local: si el equipo local es difícil según la tabla general,
        toma valor igual a la ronda. Si no lo es, toma np.nan

        ronda_f_visita: si el equipo visitante es fácil según la tabla de visita,
        toma valor igual a la ronda. Si no lo es, toma np.nan
        ronda_d_visita: si el equipo visitante es difícil según la tabla de visita,
        toma valor igual a la ronda. Si no lo es, toma np.nan    
        ronda_f_local: si el equipo local es fácil según la tabla de local,
        toma valor igual a la ronda. Si no lo es, toma np.nan
        ronda_d_local: si el equipo local es difícil según la tabla de local,
        toma valor igual a la ronda. Si no lo es, toma np.nan


        """
    
#         df_actual['f_gral_visita'] = np.where(df_actual['Visita'].isin(equipos_5u), 1, 0)
#         df_actual['d_gral_visita'] = np.where(df_actual['Visita'].isin(equipos_5p), 1, 0)
#         df_actual['f_gral_local'] = np.where(df_actual['Local'].isin(equipos_5u), 1, 0)
#         df_actual['d_gral_local'] = np.where(df_actual['Local'].isin(equipos_5p), 1, 0)
        
        df_actual['f_elo_visita'] = np.where(df_actual['Visita'].isin(elo_5u), 1, 0)
        df_actual['d_elo_visita'] = np.where(df_actual['Visita'].isin(elo_5p), 1, 0)
        df_actual['f_elo_local'] = np.where(df_actual['Local'].isin(elo_5u), 1, 0)
        df_actual['d_elo_local'] = np.where(df_actual['Local'].isin(elo_5p), 1, 0)  

#         df_actual['f_visita'] = np.where(df_actual['Visita'].isin(visita_5u), 1, 0)
#         df_actual['d_visita'] = np.where(df_actual['Visita'].isin(visita_5p), 1, 0)
#         df_actual['f_local'] = np.where(df_actual['Local'].isin(local_5u), 1, 0)
#         df_actual['d_local'] = np.where(df_actual['Local'].isin(visita_5p), 1, 0)

#         df_actual['f_gral_visita_corr'] =  df_actual['f_gral_visita']*np.where(df_actual['Local'].isin(equipos_5u),pto_corte/(pto_corte - 1) , 1)
#         df_actual['d_gral_visita_corr'] =  df_actual['d_gral_visita']*np.where(df_actual['Local'].isin(equipos_5p),pto_corte/(pto_corte - 1) , 1)
#         df_actual['f_gral_local_corr'] =  df_actual['f_gral_local']*np.where(df_actual['Visita'].isin(equipos_5u),pto_corte/(pto_corte - 1) , 1)
#         df_actual['d_gral_local_corr'] =  df_actual['d_gral_local']*np.where(df_actual['Visita'].isin(equipos_5p),pto_corte/(pto_corte - 1) , 1)

#         df_actual['f_visita_corr'] =  df_actual['f_visita']*np.where(df_actual['Local'].isin(visita_5u),pto_corte/(pto_corte - 1) , 1)
#         df_actual['d_visita_corr'] =  df_actual['d_visita']*np.where(df_actual['Local'].isin(visita_5p),pto_corte/(pto_corte - 1) , 1)
#         df_actual['f_local_corr'] = df_actual['f_local']*np.where(df_actual['Visita'].isin(local_5u),pto_corte/(pto_corte - 1) , 1)
#         df_actual['d_local_corr'] =  df_actual['d_local']*np.where(df_actual['Visita'].isin(local_5p),pto_corte/(pto_corte - 1) , 1)

#         df_actual['posicion_gral_visita'] = df_actual['Visita'].map(dict_eq_pos_gral)
#         df_actual['posicion_gral_local'] = df_actual['Local'].map(dict_eq_pos_gral)
#         df_actual['posicion_visita'] = df_actual['Visita'].map(dict_eq_pos_visita)
#         df_actual['posicion_local'] = df_actual['Local'].map(dict_eq_pos_local)


#         df_actual['ronda_f_gral_visita'] = df_actual['Round']*np.where(df_actual['Visita'].isin(equipos_5u), 1, np.nan)
#         df_actual['ronda_d_gral_visita'] = df_actual['Round']*np.where(df_actual['Visita'].isin(equipos_5p), 1, np.nan)
#         df_actual['ronda_f_gral_local'] = df_actual['Round']*np.where(df_actual['Local'].isin(equipos_5u), 1, np.nan)
#         df_actual['ronda_d_gral_local'] = df_actual['Round']*np.where(df_actual['Local'].isin(equipos_5p), 1, np.nan)

#         df_actual['ronda_f_visita'] = df_actual['Round']*np.where(df_actual['Visita'].isin(visita_5u), 1, np.nan)
#         df_actual['ronda_d_visita'] = df_actual['Round']*np.where(df_actual['Visita'].isin(visita_5p), 1, np.nan)
#         df_actual['ronda_f_local'] = df_actual['Round']*np.where(df_actual['Local'].isin(local_5u), 1, np.nan)
#         df_actual['ronda_d_local'] = df_actual['Round']*np.where(df_actual['Local'].isin(local_5p), 1, np.nan)

#         df_actual['ronda_f_gral_visita'] = df_actual['Round']*np.where(df_actual['Visita'].isin(equipos_5u), 1, np.nan)
#         df_actual['ronda_d_gral_visita'] = df_actual['Round']*np.where(df_actual['Visita'].isin(equipos_5p), 1, np.nan)
#         df_actual['ronda_f_gral_local'] = df_actual['Round']*np.where(df_actual['Local'].isin(equipos_5u), 1, np.nan)
#         df_actual['ronda_d_gral_local'] = df_actual['Round']*np.where(df_actual['Local'].isin(equipos_5p), 1, np.nan)

#         df_actual['ronda_f_visita'] = df_actual['Round']*np.where(df_actual['Visita'].isin(visita_5u), 1, np.nan)
#         df_actual['ronda_d_visita'] = df_actual['Round']*np.where(df_actual['Visita'].isin(visita_5p), 1, np.nan)
#         df_actual['ronda_f_local'] = df_actual['Round']*np.where(df_actual['Local'].isin(local_5u), 1, np.nan)
#         df_actual['ronda_d_local'] = df_actual['Round']*np.where(df_actual['Local'].isin(local_5p), 1, np.nan)
        
#         for r in [4,5,6,7]:
        for r in [3,4,5,6]:
            df_efecto = variables_efecto_secuencial(df_actual, punto_corte = c, ronda_corte = r)
            if r == 3:
#                 conditions_g = [
#                         (df_efecto['equipo'].isin(equipos_5p)),
#                         (df_efecto['equipo'].isin(equipos_5u))]
#                 conditions_l = [
#                         (df_efecto['equipo'].isin(local_5p)),
#                         (df_efecto['equipo'].isin(local_5u))]
#                 conditions_v = [
#                         (df_efecto['equipo'].isin(visita_5p)),
#                         (df_efecto['equipo'].isin(visita_5u))]
                conditions_e = [
                        (df_efecto['equipo'].isin(elo_5p)),
                        (df_efecto['equipo'].isin(elo_5u))]
#                 conditions = [conditions_g, conditions_l, conditions_v, conditions_e]
                conditions = [conditions_e]
                choices = ['dificil','facil']
#                 newvar_names = ['categoria_general_c%s' % c,
#                                 'categoria_local_c%s' % c,
#                                 'categoria_visita_c%s' % c,
#                                 'categoria_elo_c%s' % c]
                newvar_names = ['categoria_elo_c%s' % c]
                for n in range(len(newvar_names)):
                    newvar = newvar_names[n]    
                    condition = conditions[n]
                    df_efecto[newvar] = np.select(condition,
                                                  choices,
                                                  default = 'regular')
                    df_efecto[newvar] = df_efecto[newvar].astype('category')
                    try:
                        df_efecto[newvar].cat.reorder_categories(['regular', 'facil','dificil'], inplace = True)
                    except:
                        print('Torneo: %s' % torneo_actual, df_efecto[newvar].drop_duplicates().tolist())
                if c == 3:
                    dfs_efecto.append(df_efecto)
                else:
                    dfs_efecto.append(df_efecto[[i for i in df_efecto.columns if i not in cols_main]])
            else:
                dfs_efecto.append(df_efecto[[i for i in df_efecto.columns if i not in cols_main]])
            
            
    df_efecto_final = pd.concat(dfs_efecto, axis = 1)
    return df_efecto_final

In [9]:
def indicadores_liga(df, df_elo):
    df = df.sort_values(by=['Date','Round'], ascending = [True, True]).reset_index(drop=True)
    Torneos = df['Torneo'].drop_duplicates().tolist()
    dfs_efecto = []
    for i in range(1,len(Torneos)):
        dfs_efecto.append(indicadores_torneo(df,df_elo,Torneos[i],Torneos[i-1]))
    df_efecto = pd.concat(dfs_efecto, axis = 0, ignore_index = True)
    return df_efecto

In [10]:
def bases_efecto_secuencial(datadir,outputdir):
    for subdir, dirs, files in os.walk(datadir):
        for file in files:
            filepath = os.path.join(subdir,file)
            if filepath.endswith(".xlsx") and 'Random' not in filepath:
                sheet = file.replace('.xlsx','')
                print('Generando: % s           ' % sheet, end = '\t\r')
                df = pd.read_excel(filepath)
                df_efecto = indicadores_liga(df)
                df_efecto.to_excel(os.path.join(outputdir, file), sheet_name = sheet, index = False)

## Ejecución

In [ ]:
%%time
datadir = os.path.join(os.path.pardir, 'datos','ligas-hist')
elodir = os.path.join(os.path.pardir, 'datos','elo')
outputdir = os.path.join(os.path.pardir, 'datos','regresiones')
ligas = ['Espana']
# ligas =['Inglaterra']
# df_premier = pd.read_excel(os.path.join(outputdir,'Inglaterra.xlsx'))
# columns = df_premier.columns.tolist()
# columns = columns + []
for liga in ligas:
    archivo = liga + '.xlsx'
    archivo_ = liga + '-hist.xlsx'
    df_test = pd.read_excel(os.path.join(datadir,archivo))
    df_test = df_test[df_test['Date'] > '1950-05-31'].reset_index(drop=True)
    df_elo = pd.read_excel(os.path.join(elodir,archivo_))
    df_efecto_test = indicadores_liga(df_test, df_elo)
    df_efecto_test = df_efecto_test.loc[:,~df_efecto_test.columns.duplicated()]
    df_efecto_test = df_efecto_test[[i for i in df_efecto_test.columns if 'corr' not in i and 'regular' not in i]]
#     print(df_efecto_test.columns.tolist())
    df_efecto_test.to_excel(os.path.join(outputdir, archivo_), sheet_name = liga, index = False)

In [12]:
# %%time
# bases_efecto_secuencial(datadir,outputdir)

In [13]:
# datadir = os.path.join(os.path.pardir, 'Datos','Bases construidas','Bases main', 'Bases efecto')
# df_test = pd.read_excel(os.path.join(datadir,'Chile.xlsx'))

In [14]:
# df_test.head()